In [1]:
import wget
import bz2
import numpy as np
import os 

In [2]:
wget.download("ftp://hmwr829gr.cr.chiba-u.ac.jp/gridded/FD/V20151105/201507/TIR/201507070000.tir.01.fld.geoss.bz2")

fname = "201507070000.tir.01.fld.geoss.bz2"
dbuf = bz2.BZ2File(fname).read()
dataDN = np.frombuffer(dbuf,dtype='>u2').reshape(6000,6000)
os.remove(fname)
dataDN

100% [....................................................................] 27465961 / 27465961

array([[2862, 2862, 2862, ..., 3165, 3165, 3165],
       [2775, 2775, 2775, ..., 3235, 3235, 3176],
       [2961, 2949, 2949, ..., 3166, 3166, 3166],
       ...,
       [3544, 3561, 3561, ..., 2947, 2947, 2947],
       [3518, 3518, 3534, ..., 3120, 3120, 2834],
       [3534, 3534, 3534, ..., 2947, 2947, 2947]], dtype='>u2')

### look-up table

In [4]:
import wget
import tarfile

wget.download('ftp://hmwr829gr.cr.chiba-u.ac.jp/gridded/FD/support/count2tbb_v102.tgz')
# 解凍
with tarfile.open('count2tbb_v102.tgz', 'r:gz') as t:
    t.extractall(path='src')
%cd src
DN,tbb = np.loadtxt('tir.01', unpack = True)
print(DN,tbb)
# Tbb 変換
dataTBB = tbb[dataDN]
dataTBB

100% [......................................................................] 1055494 / 1055494C:\Users\mujina\Documents\本地\観天望気\衛星画像\Himawari\src
[0.000e+00 1.000e+00 2.000e+00 ... 4.048e+03 4.049e+03 4.050e+03] [330.966134 330.946875 330.927613 ... 117.385241 110.857281  69.998635]


C:\Users\mujina\AppData\Local\Programs\Python\Python312\Lib\site-packages\IPython\core\magics\osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


array([[256.374486, 256.374486, 256.374486, ..., 243.151015, 243.151015,
        243.151015],
       [259.762674, 259.762674, 259.762674, ..., 239.688749, 239.688749,
        242.619123],
       [252.321123, 252.824658, 252.824658, ..., 243.10284 , 243.10284 ,
        243.10284 ],
       ...,
       [221.434605, 220.231344, 220.231344, ..., 252.908238, 252.908238,
        252.908238],
       [223.222653, 223.222653, 222.129561, ..., 245.283125, 245.283125,
        257.481606],
       [222.129561, 222.129561, 222.129561, ..., 252.908238, 252.908238,
        252.908238]])

## TBB を xarray に変換する

In [6]:
import xarray as xr
# data に対応する緯度経度を定義する
lon1d = np.arange(85.0, 205.0, 0.02).astype(np.float64)
lat1d = np.arange(60.0, -60.0, -0.02).astype(np.float64)
xr_dataTBB = xr.DataArray(np.float32(dataTBB), name='Himawari-8',\
        coords={
        'lat':('lat',lat1d,{'units':'Degrees_North'}),
        'lon':('lon',lon1d,{'units':'Degrees_East'})},
        dims=['lat','lon'])
xr_dataTBB 

<xarray.DataArray 'Himawari-8' (lat: 6000, lon: 6000)> Size: 144MB
array([[256.37448, 256.37448, 256.37448, ..., 243.15102, 243.15102,
        243.15102],
       [259.76266, 259.76266, 259.76266, ..., 239.68875, 239.68875,
        242.61913],
       [252.32112, 252.82466, 252.82466, ..., 243.10284, 243.10284,
        243.10284],
       ...,
       [221.4346 , 220.23134, 220.23134, ..., 252.90823, 252.90823,
        252.90823],
       [223.22266, 223.22266, 222.12956, ..., 245.28313, 245.28313,
        257.4816 ],
       [222.12956, 222.12956, 222.12956, ..., 252.90823, 252.90823,
        252.90823]], dtype=float32)
Coordinates:
  * lat      (lat) float64 48kB 60.0 59.98 59.96 59.94 ... -59.94 -59.96 -59.98
  * lon      (lon) float64 48kB 85.0 85.02 85.04 85.06 ... 204.9 205.0 205.0

- データは xr_dataTBB として xarray に変換された．このデータを PyGMT で読み込み，map 上にひまわりデータを表示してみる．

In [8]:
import pygmt

fig = pygmt.Figure()
# set the frame solid line
pygmt.config(MAP_FRAME_TYPE="plain")
fig.basemap(region=[122.5, 154, 20, 48], projection="Q15c", frame=True)
with pygmt.config(COLOR_FOREGROUND="red", COLOR_BACKGROUND="black"):
    pygmt.makecpt(cmap="gray", series=[190, 300, 10], continuous=True, M=True, reverse=True,)
fig.grdimage(grid=xr_dataTBB, projection="Q15c")
fig.colorbar(frame=["a10", "x+lTBB", "y+l(K)"])
fig.coast(resolution="h", shorelines="1/1p,green")
fig.basemap(frame=["ag", "+tHimawari-8_Band13"])
fig.text(text="20150707 00:00", x=128, y=22, font="12p,Helvetica-Bold,blue", angle=0, fill="gray")
fig.savefig("Himawari_tbb.png")
fig.show()

GMTCLibNotFoundError: Error loading GMT shared library at 'gmt.dll'.
Could not find module 'gmt.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Error loading GMT shared library at 'gmt_w64.dll'.
Could not find module 'gmt_w64.dll' (or one of its dependencies). Try using the full path with constructor syntax.
Error loading GMT shared library at 'gmt_w32.dll'.
Could not find module 'gmt_w32.dll' (or one of its dependencies). Try using the full path with constructor syntax.

## Missing GMT 本体 (gmt.dll)